# The demo file of unfoledVBA for RGB images

In [1]:
import os
import matplotlib.pyplot as plt
import torch
import numpy as np
import scipy.io as sio
%matplotlib inline
import time
import random
from Model_files.unfoldedVBA_vec_RGB import VBA_class
from Model_files.tools import *
from Model_files.model_vec import *
from Model_files.initializationdic_RGB import *
import cv2
import traceback

Loading Test set
Loading Train set
Loading Train RGB set
Loading Train RGB set
Loading Test RGB set


In [2]:
torch.cuda.is_available()

True

## 1- Test saved models
### 1.1- Test conditions

In [3]:
dataset         = 'Flickr30'  
name_kernel     = 'Mixed' 
noise_std_min   = 0.005 
noise_std_max   = 0.05 
noise_std_range = [noise_std_min, noise_std_max] # one random noise std
device = 'GPU'

In [4]:
im_size         = (256,256)
im_range        = [0,1]       # minimal and maximal pixel values
test_conditions = [name_kernel, noise_std_range, im_size, im_range]

# Path to the train/val/test set and to the folder with the saved model
path_trainset      = 'Datasets/Trainsets_RGB'
path_valset        = 'Datasets/Valsets_RGB'
path_testset       = 'Datasets/Testsets_RGB'
path_save          = os.path.join('Trainings',name_kernel,'saved_model')
paths              = [path_testset,path_trainset,path_valset,path_save]

print('--------------------------------------------------------------------------------------------------------------------------------')
print('Testset: %s'%(dataset))
print('Blur   : %s'%(name_kernel))
print('--------------------------------------------------------------------------------------------------------------------------------')

--------------------------------------------------------------------------------------------------------------------------------
Testset: Flickr30
Blur   : Mixed
--------------------------------------------------------------------------------------------------------------------------------


### 1.1.1- Change RGB image $\in$ [0,1]

In [ ]:
path_rgb = 'Datasets/Groundtruth/full/' + dataset
path_rgb_nor = 'Datasets/Groundtruth/full1_RGB/'+ dataset# normalized RGB image saved here
# create the path if it does not exist 
if not os.path.exists(path_rgb_nor):
    os.makedirs(path_rgb_nor)

In [ ]:
for img_name in os.listdir(os.path.join(path_rgb)):
    if img_name =='.ipynb_checkpoints': continue
    path_read = os.path.join(path_rgb,img_name)
    image = cv2.cvtColor(cv2.imread(path_read), cv2.COLOR_BGR2RGB)
    nor_image = (image-image.min())/(image.max()-image.min()) # the normalization of the image
    path_save = os.path.join(path_rgb_nor,str(os.path.splitext(img_name)[0])+'.mat')
    sio.savemat(path_save, {'image': nor_image})    

### 1.2- Create blurred test images

In [ ]:
size_testset = len([n for n in os.listdir(os.path.join('Datasets/Groundtruth/full1_RGB',dataset))]) # number of test images

In [ ]:
print('The size of the test set is',size_testset)

In [ ]:
print('--------------------------------------------------------------------------------------------------------------------------------',flush=True)
print('Creating %d blurred test images from %s using %s blur kernel...'%(size_testset,dataset,name_kernel),flush=True)
create_testset_RGB(dataset,'Datasets/Groundtruth',path_testset,name_kernel,noise_std_range,im_size)
print('--------------------------------------------------------------------------------------------------------------------------------',flush=True)

### 1.3- Load a saved model

In [ ]:
# choose the model learned by greedy approach or N-N training
saved_model = 'greedy_approach'
#saved_model = 'N-N training'

In [ ]:
if saved_model == 'greedy_approach':
    path_save_block = os.path.join('Trainings',name_kernel,'final_model_greedy_block22')
else:
    path_save_block = os.path.join('Trainings',name_kernel,'final_model_N_N_bloc22')
paths              = [path_testset,path_trainset,path_valset,path_save] 

In [ ]:
network              = VBA_class(test_conditions, paths, mode='test',device = device)
path_layers          = os.path.join(path_save_block,'trained_model_MinLossOnVal.pt')
path_post_processing = os.path.join(path_save_block,'trained_post-processing_MinLossOnVal.pt')
network.model.load_state_dict(torch.load(path_layers))
network.last_layer.load_state_dict(torch.load(path_post_processing))
if device == 'CPU': 
    network.to('cpu')  
print('--------------------------------------------------------------------------------------------------------------------------------',flush=True)
print('Loaded unfoldedVBA layers from %s.'%(path_layers))
print('Loaded the post-processing layer from %s.'%(path_post_processing))
print('--------------------------------------------------------------------------------------------------------------------------------',flush=True)

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in network.model.state_dict():
    print(param_tensor, "\t", network.model.state_dict()[param_tensor])

In [ ]:
k = 0
i = 11
no_param = 21#block
for f in network.parameters():
    if k == no_param*i:
        conv3_weight = f.data
        print('conv3_weight is',conv3_weight)
    elif k == no_param*i+1:
        #print('This is',k)
        conv3_bias = f.data
    elif k == no_param*i+2:
        #print('This is',k)
        conv2_weight = f.data
    elif k == no_param*i+3:
        #print('This is',k)
        conv2_bias = f.data
    elif k == no_param*i+4:
        #print('This is',k)
        lin1_weight = f.data
    elif k == no_param*i+5:
        #print('This is',k)
        lin1_bias = f.data
    elif k == no_param*i+6:
        #print('This is',k)
        lin3_weight = f.data
    elif k == no_param*i+7:
        #print('This is',k)
        lin3_bias = f.data
    elif k == no_param*i+8:
        #print('This is',k)
        a_k = f.data
    elif k == no_param*i+9:
        #print('This is',k)
        b_k = f.data 
    elif k == no_param*i+10:
        #print('This is',k)
        Haar = f.data     
    k += 1    

In [ ]:
block5 = {}
block5['conv3_weight'] = conv3_weight
block5['conv3_bias'] = conv3_bias
block5['conv2_weight'] = conv2_weight
block5['conv2_bias'] = conv2_bias
block5['lin1_weight'] = lin1_weight
block5['lin1_bias'] = lin1_bias
block5['lin3_weight'] = lin3_weight
block5['lin3_bias'] = lin3_bias
block5['a_k'] = a_k
block5['b_k'] = b_k
block5['Haar'] = Haar

In [ ]:
# save the model
import pickle
with open('block0_greedy_RGB.txt', 'wb') as handle:
    pickle.dump(block5, handle)




### 1.4- Test a model

In [ ]:
print('--------------------------------------------------------------------------------------------------------------------------------',flush=True)
print('Testing model %s ...'%(path_save),flush=True)
time_start = time.time()
network.test(dataset)
time_per_image = (time.time()-time_start)/size_testset
print('Average time per image: %.2f seconds'%(time_per_image),flush=True)
print('--------------------------------------------------------------------------------------------------------------------------------',flush=True)

## 2- Train a model
### 2.1- Change RGB $\in$ [0,1] for training and validation set

In [ ]:
# path_rgb = 'Datasets/BSD500_COCO1000_train_val/train'
# path_rgb_nor = 'Datasets/BSD500_COCO1000_train_val/train1_RGB'
# # create the path if it does not exist 
# if not os.path.exists(path_rgb_nor):
#     os.makedirs(path_rgb_nor)

path_rgb = 'Datasets/BSD500_COCO1000_train_val/val'
path_rgb_nor = 'Datasets/BSD500_COCO1000_train_val/val1_RGB'
# create the path if it does not exist 
if not os.path.exists(path_rgb_nor):
    os.makedirs(path_rgb_nor)

for img_name in os.listdir(path_rgb):
    if img_name != '.ipynb_checkpoints':
        path_read = os.path.join(path_rgb,img_name)
        image = cv2.cvtColor(cv2.imread(path_read), cv2.COLOR_BGR2RGB)
        nor_image = (image-image.min())/(image.max()-image.min()) # the normalization of the image
        #print('true image is ',nor_image)
        path_save = os.path.join(path_rgb_nor,str(os.path.splitext(img_name)[0])+'.mat')
        sio.savemat(path_save, {'image': nor_image})  

In [ ]:
# path_rgb = 'Datasets/BSD500_COCO1000_train_val/train_RGB'
# path_gray = 'Datasets/BSD500_COCO1000_train_val/train1_RGB'

path_rgb = 'Datasets/BSD500_COCO1000_train_val/val_RGB'
path_gray = 'Datasets/BSD500_COCO1000_train_val/val1_RGB'

In [ ]:
size_trainset = len([n for n in os.listdir(os.path.join(path_gray)) if n != '.ipynb_checkpoints']) # number of test images

In [ ]:
print('The size of the train set is',size_trainset)

### 2.2- Center crop image pairs for training and validation and save

In [ ]:
path_groundtruth_train = 'Datasets/BSD500_COCO1000_train_val/train1_RGB'
path_trainset = 'Datasets/Trainsets_RGB'
# create the path if it does not exist
if not os.path.exists(path_trainset):
    os.makedirs(path_trainset)
im_size         = (256,256)
create_trainset_RGB(path_groundtruth_train,path_trainset,noise_std_range,im_size)    

path_groundtruth_val = 'Datasets/BSD500_COCO1000_train_val/val1_RGB'
path_valset = 'Datasets/Valsets_RGB'
# create the path if it does not exist
if not os.path.exists(path_valset):
    os.makedirs(path_valset)
im_size         = (256,256)
create_trainset_RGB(path_groundtruth_val,path_valset,noise_std_range,im_size)

In [ ]:
size_trainset = len([n for n in os.listdir(os.path.join(path_trainset))]) # number of train images
size_valset = len([n for n in os.listdir(os.path.join(path_valset))]) # number of test images

In [ ]:
print('The size of the train set is',size_trainset)
print('The size of the val set is',size_valset)

In [ ]:
data= sio.loadmat('Datasets/Trainsets_RGB/000000001072_blur11.mat')
#data= sio.loadmat('Datasets/Valsets_RGB/101085_blur14.mat')
image = data['image']
trueimage = data['trueimage']
h = data['h']
noise_std = data['noise_std']
plt.figure()
plt.imshow(h,cmap='gray')
plt.show()

### 2.3- Training parameters
##### Please refer to unfoldedVBA_vec_RGB.py to see all training parameters (learning rates, batch size, loss function, etc...)

In [10]:
lr_first_layer     = 5e-3  #learning rate to train the first layer
lr_greedy          = 1e-3   #learning rate to train the following layers during greedy approach
lr_lpp             = 1e-3   #learning rate to train the post-processing layer
lr_N_N             = 5e-5  #learning rate to train all the layers together + lpp during greedy approach
mode               ='first_layer'    
nb_epochs          = [1,1,2,2] #number of epochs for training the first layer, the remaining layers, the post-processing layer, N-N training respectively
network            = VBA_class(test_conditions, paths, mode,
                         lr_first_layer     = lr_first_layer,
                         lr_greedy          = lr_greedy,
                         lr_lpp             = lr_lpp,
                         lr_N_N             = lr_N_N,
                         nb_epochs          = nb_epochs,
                         device             = device)

### 2.4- Train a model

In [11]:
random.seed(a=1, version=2)

In [12]:
init(network) # use the initial parameter for the first layer
if device == 'CPU': 
    network.to('cpu')

conv3_weight is tensor([[[[-2.6845, -4.7743, -4.0976],
          [-3.1357, -4.4324, -2.4751],
          [-2.2557, -1.6146,  0.5117]]]], device='cuda:0')


In [ ]:
#This line opens a log file
with open("bug_log.txt", "w") as log:

    try:
        network.train()
        print("There is no bug.", file = log)
    except Exception:
        traceback.print_exc(file=log) 